In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [5]:
features_list=[
'bounced_transactions_count',
'utilities_spending_pct',
'min_balance_last_3_months',
'num_debits_last_1_months',
'days_balance_lt_5000_M3',
'num_credit_gt_salary',
'cash_withdrawal_sum',
'days_balance_lt_10000_M3',
'ratio_balance_after_7_days_max_credit_M1',
'ratio_5th_day_balance_to_max_credit_M2',
'Max_Credit_Min_M1_M2_M3',
'sum_insurance_debit',
'balance_after_7_days_max_credit_M3',
'spend_vs_salary_ratio',
'month_year',
 '90_in_24mob']

In [6]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3024070/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3024070/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 14), (15604, 14), (23406,), (15604,))

In [9]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 14) (4682, 14) (15604, 14)


In [10]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)

In [11]:


params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
    'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 12.6,
    'alpha': 4.8,
    'gamma': 5.6,
    'random_state': 42   
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 100,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.54020	eval-auc:0.51918
[10]	train-auc:0.55661	eval-auc:0.53066
[20]	train-auc:0.55925	eval-auc:0.53577
[30]	train-auc:0.56999	eval-auc:0.54368
[40]	train-auc:0.57190	eval-auc:0.54616
[50]	train-auc:0.57280	eval-auc:0.54684
[60]	train-auc:0.57343	eval-auc:0.54968
[70]	train-auc:0.57253	eval-auc:0.54733
[80]	train-auc:0.57252	eval-auc:0.54449
[90]	train-auc:0.57180	eval-auc:0.54974
[99]	train-auc:0.57369	eval-auc:0.54758


In [12]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.57, Gini: 0.15, KS: 0.09
Eval AUC: 0.55, Gini: 0.10, KS: 0.06
Test AUC: 0.55, Gini: 0.11, KS: 0.07
OOT AUC: 0.56, Gini: 0.13, KS: 0.10


In [13]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
6          0.272824         0.225687        0.267848        0.205434   1873   
5          0.191239         0.188972        0.205429        0.180153   1872   
4          0.194139         0.179632        0.180151        0.179129    273   
3          0.163489         0.179095        0.179122        0.177807  10961   
2          0.146681         0.175256        0.177804        0.173633   1868   
1          0.086308         0.170793        0.173632        0.167051   1877   

        label_good  label_bad  
Decile                         
6             1362        511  
5             1514        358  
4              220         53  
3             9169       1792  
2             1594        274  
1             1715        162

In [14]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
6          0.237668         0.225330        0.267688        0.203357   1561   
5          0.197436         0.187704        0.203294        0.179446   1560   
4          0.226190         0.179284        0.179439        0.179125     84   
3          0.167277         0.179109        0.179122        0.178262   9278   
2          0.162776         0.175655        0.178260        0.173897   1542   
1          0.105130         0.171189        0.173896        0.167051   1579   

        label_good  label_bad  
Decile                         
6             1190        371  
5             1252        308  
4               65         19  
3             7726       1552  
2             1291        251  
1             1413        166

In [15]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
6          0.250618         0.230107        0.265068        0.210766   1213   
5          0.218467         0.194024        0.210732        0.182683   1213   
4          0.199095         0.180663        0.182670        0.179129    442   
3          0.161838         0.179100        0.179122        0.177866   6834   
2          0.146744         0.175400        0.177858        0.173679   1213   
1          0.122012         0.170919        0.173679        0.167051   1213   

        label_good  label_bad  
Decile                         
6              909        304  
5              948        265  
4              354         88  
3             5728       1106  
2             1035        178  
1             1065        148

In [16]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                     Feature  Importance
0                 bounced_transactions_count        52.0
6                        cash_withdrawal_sum        52.0
5                       num_credit_gt_salary        22.0
1                     utilities_spending_pct        18.0
2                  min_balance_last_3_months        18.0
4                    days_balance_lt_5000_M3        16.0
7                   days_balance_lt_10000_M3        12.0
13                     spend_vs_salary_ratio        11.0
10                   Max_Credit_Min_M1_M2_M3        10.0
3                   num_debits_last_1_months         9.0
12        balance_after_7_days_max_credit_M3         5.0
9     ratio_5th_day_balance_to_max_credit_M2         2.0
11                       sum_insurance_debit         2.0
8   ratio_balance_after_7_days_max_credit_M1         1.0


In [17]:
feature_importance_df

Feature  Importance
0                 bounced_transactions_count        52.0
6                        cash_withdrawal_sum        52.0
5                       num_credit_gt_salary        22.0
1                     utilities_spending_pct        18.0
2                  min_balance_last_3_months        18.0
4                    days_balance_lt_5000_M3        16.0
7                   days_balance_lt_10000_M3        12.0
13                     spend_vs_salary_ratio        11.0
10                   Max_Credit_Min_M1_M2_M3        10.0
3                   num_debits_last_1_months         9.0
12        balance_after_7_days_max_credit_M3         5.0
9     ratio_5th_day_balance_to_max_credit_M2         2.0
11                       sum_insurance_debit         2.0
8   ratio_balance_after_7_days_max_credit_M1         1.0

In [18]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BA_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BA_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BA_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BA_Oot_Pred_9024.csv',index=False)


In [19]:
#Save the models
xgb_model.save_model('LeoPayu_BA_90_in_24mob.json')
print('Model Saved')

Model Saved
